In [1]:
import tensorflow as tf
from tensorflow import keras
import json
import numpy as np
import os  
import pandas as pd

2022-12-29 16:13:02.467560: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-29 16:13:09.971911: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-29 16:13:09.971952: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-29 16:13:25.321172: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']= '-1'
sentences = []
labels = []
urls = []    
datastore = pd.read_json('/home/nasir/Desktop/python practice/data/archive(1)/Sarcasm_Headlines_Dataset_v2.json',lines=True)
datastore.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [3]:
sentences = datastore['headline']
urls = datastore['article_link']
labels = datastore['is_sarcastic']

In [4]:
tokenzer = keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
tokenzer.fit_on_texts(sentences)
word_index = tokenzer.word_index

In [5]:
sequences = tokenzer.texts_to_sequences(sentences)
padding = keras.preprocessing.sequence.pad_sequences(sequences,padding='post')
print(padding[0])
print(padding.shape)

[16004   355  3167  7474  2644     3   661  1119     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
(28619, 152)


In [6]:
training_size = 20000
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_label = labels[0:training_size]
testing_label = labels[training_size:]

In [7]:
vocab_size = 10000
max_lenght = 100
embedding_dim = 16
tokenzer =  keras.preprocessing.text.Tokenizer(num_words= vocab_size,oov_token='<OOV>' )
tokenzer.fit_on_texts(training_sentences)
word_index = tokenzer.word_index

In [8]:
sequences = tokenzer.texts_to_sequences(training_sentences)
training_padding = keras.preprocessing.sequence.pad_sequences(sequences,maxlen=max_lenght,padding='post',truncating='post')
testing_sequences = tokenzer.texts_to_sequences(testing_sentences)
testing_padding =  keras.preprocessing.sequence.pad_sequences(testing_sequences,maxlen=max_lenght,padding='post',truncating='post')

In [23]:
model = keras.Sequential([keras.layers.Embedding(vocab_size,embedding_dim,input_length = max_lenght),keras.layers.GlobalAveragePooling1D(),keras.layers.Dense(24,activation = 'relu'),keras.layers.Dense(1,activation = 'sigmoid')])


In [26]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [28]:
model.fit(training_padding,training_label,epochs=20,validation_data=(testing_padding,testing_label))

Epoch 1/20
625/625 [==============================] - 3s 5ms/step - loss: 0.1130 - acc: 0.9610 - val_loss: 0.4406 - val_acc: 0.8386
Epoch 2/20
625/625 [==============================] - 3s 5ms/step - loss: 0.1037 - acc: 0.9652 - val_loss: 0.4675 - val_acc: 0.8344
Epoch 3/20
625/625 [==============================] - 4s 6ms/step - loss: 0.0936 - acc: 0.9689 - val_loss: 0.4978 - val_acc: 0.8320
Epoch 4/20
625/625 [==============================] - 4s 6ms/step - loss: 0.0859 - acc: 0.9719 - val_loss: 0.5272 - val_acc: 0.8296
Epoch 5/20
625/625 [==============================] - 3s 4ms/step - loss: 0.0789 - acc: 0.9740 - val_loss: 0.5805 - val_acc: 0.8254
Epoch 6/20
625/625 [==============================] - 2s 4ms/step - loss: 0.0735 - acc: 0.9758 - val_loss: 0.5908 - val_acc: 0.8249
Epoch 7/20
625/625 [==============================] - 2s 4ms/step - loss: 0.0674 - acc: 0.9781 - val_loss: 0.6272 - val_acc: 0.8238
Epoch 8/20
625/625 [==============================] - 2s 4ms/step - loss: 0.

In [29]:
sentence = ['This thing does not work','you think, you are smart','holy cow, you think you smart']
new_sequence = tokenzer.texts_to_sequences(sentence)
new_padd = keras.preprocessing.sequence.pad_sequences(new_sequence,maxlen=max_lenght,padding='post',truncating='post')

In [30]:
print(model.predict(new_padd))

1/1 [==============================] - 0s 105ms/step
[[0.01345476]
 [0.03960415]
 [0.869339  ]]
